# Data

In [ ]:
import albumentations as A
import albumentations.augmentations.geometric.functional as F
import cv2

class AspectRatioResize(A.DualTransform):
    """Resize the input to the given height and width.

    Args:
        p (float): probability of applying the transform. Default: 1.
        height (int): desired height of the output.
        width (int): desired width of the output.
        interpolation (OpenCV flag): flag that is used to specify the interpolation algorithm. Should be one of:
            cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4.
            Default: cv2.INTER_LINEAR.

    Targets:
        image, mask, bboxes

    Image types:
        uint8, float32
    """

    def __init__(self, height, width, interpolation=cv2.INTER_LINEAR, always_apply=False, p=1):
        super(AspectRatioResize, self).__init__(always_apply, p)
        self.height = height
        self.width = width
        self.interpolation = interpolation

    def apply(self, img, interpolation=cv2.INTER_LINEAR, **params):
        height, width, _ = img.shape
        r = min(self.width / width, self.height / height)
        return F.resize(
            img, height=int(r * height), width=int(r * width), interpolation=interpolation
        )

    def apply_to_bbox(self, bbox, **params):
        # Bounding box coordinates are scale invariant
        return bbox

In [ ]:
import albumentations as A

IMAGE_SIZE = (1088, 608)

test_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0
)

## Crowd human

In [ ]:
from pathlib import Path

crowd_data_dir = Path("/home/jasseur/Downloads/crowdhuman")

In [ ]:
import pandas as pd

crowd_test_file = Path("../../../data/crowdhuman/annotation_val_with_classes.csv")
crowd_test_df = pd.read_csv(crowd_test_file)
crowd_test_df = pd.DataFrame(crowd_test_df["id"].unique(), columns=["id"])
crowd_test_df.head(1)

In [ ]:
import pandas as pd
from src.test_dataset import TestDataset
%load_ext autoreload
%autoreload 2

crowd_test_dir = Path(crowd_data_dir, "Images")
crowd_test_dataset = TestDataset(crowd_test_dir, crowd_test_df, test_transform)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import cv2

image_name, image, image_size = crowd_test_dataset[0].values()

plt.figure()
plt.imshow(image)
plt.show()

## Khaliji

In [ ]:
from pathlib import Path

khaliji_data_dir = Path("../../../data/khaliji")
!ls $khaliji_data_dir

In [ ]:
import pandas as pd
from src.test_dataset import TestDataset
%load_ext autoreload
%autoreload 2

khaliji_test_dir = Path(khaliji_data_dir, "Images")
khaliji_test_df = pd.DataFrame(
    list(image_path.stem for image_path in khaliji_test_dir.iterdir()),
    columns=["id"]
)
khaliji_test_dataset = TestDataset(khaliji_test_dir, khaliji_test_df, test_transform)

In [ ]:
len(khaliji_test_dataset)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import cv2

image_name, image, image_size = khaliji_test_dataset[0].values()

plt.figure()
plt.imshow(image)
plt.show()

# Prediction

In [ ]:
from src.pose_dla_dcn import get_pose_net as get_dla_dcn
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

num_layers = 34
heads = {"hm": 1, "wh": 4, "reg": 2}
head_conv = 256
net = get_dla_dcn(num_layers, heads, head_conv)
model_path = Path("checkpoint", "10.pth")
# model_path = Path(data_dir, "fairmot_dla34.pth")
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda:1", checkpoint_dir=None, model_path=model_path
)

In [ ]:
test_df = trainer.predict(khaliji_test_dataset, batch_size=4)

In [ ]:
test_df["fbox"] = test_df[["center", "dimension"]].apply(
    lambda d: str([
        int(d["center"][0] - d["dimension"][0]),
        int(d["center"][1] - d["dimension"][1]),
        int(d["dimension"][0] + d["dimension"][2]),
        int(d["dimension"][1] + d["dimension"][3])
    ]), axis=1
)

In [ ]:
test_df.head(1)

In [ ]:
train_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=[]
    )
)

In [ ]:
from src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

train_dataset = TrainDataset(khaliji_test_dir, test_df, train_transform)

In [ ]:
len(train_dataset)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import cv2

image, centers, dimensions = train_dataset[8].values()
for center, dimension in zip(centers, dimensions):
    x, y = map(int, center)
    l, t, r, b = map(int, dimension)
    cv2.rectangle(
        image, (x - l, y - t), (x + r, y + b), (1.0, 0.0, 0.0), 2
    )

plt.figure()
plt.imshow(image)
plt.show()